In [1]:
import pandas as pd
import numpy as np

In [2]:
nrodada = 22
df = pd.read_pickle(r'.\Simulacao\simulacao_{}rodada.pkl'.format(str(nrodada)))

In [3]:
esperado = df[['Jogador','Posicao','Time','Pontuacao']].groupby(['Jogador','Posicao','Time']).agg('mean')
esperado.reset_index(inplace=True)
esperado['chave'] = esperado.apply(lambda x: '{}_{}_{}'.format(x['Time'],x['Jogador'],x['Posicao']),axis=1)

In [4]:
realizado = pd.read_pickle(r'.\Isoladas\{}rodada_isolada.pkl'.format(str(nrodada)))

In [5]:
campos_media = ['Gol', 'Assistência', 'Finalização_na_trave', 'Finalização_pra_fora', 'Roubada_de_bola', 'Passe_errado',
                'Falta_cometida', 'Pênalti_perdido', 'Gol_contra', 'Falta_sofrida','Impedimento', 'Cartão_amarelo',
                'Cartão_vermelho', 'Defesa_de_pênalti','Jogos_sem_sofrer_gols', 'Finalização_defendida',
                'Defesa_difícil', 'Gol_sofrido']

realizado['TotalQuesitos'] = realizado[campos_media].sum(axis=1)
realizado = realizado[realizado['TotalQuesitos'] != 0]
realizado.reset_index(inplace=True)
realizado.drop(['index','TotalQuesitos'],axis=1, inplace=True)

In [6]:
realizado = realizado[['Time','Apelido','Posicao','Atleta_pontos']]
realizado['chave'] = realizado.apply(lambda x: '{}_{}_{}'.format(x['Time'],x['Apelido'],x['Posicao']),axis=1)

In [7]:
esperado.head()

,Jogador,Posicao,Time,Pontuacao,chave
0,Adilson,Meia,Atlético-MG,1.34842,Atlético-MG_Adilson_Meia
1,Agenor,Goleiro,Sport,3.08906,Sport_Agenor_Goleiro
2,Airton,Meia,Botafogo,0.78640,Botafogo_Airton_Meia
3,Airton,Zagueiro,Avaí,1.89924,Avaí_Airton _Zagueiro
4,Alan Cardoso,Lateral,Vasco,-1.29366,Vasco_Alan Cardoso_Lateral


In [8]:
df = pd.merge(esperado,realizado[['chave','Atleta_pontos']],how='inner',on='chave')

In [9]:
df['Diferenca'] = df.Atleta_pontos - df.Pontuacao

In [10]:
erro_times = pd.DataFrame(df.groupby('Time').mean()['Diferenca'])
erro_times.reset_index(inplace=True)
erro_times.columns = ['Time','Erro_Medio']
#erro_times['Rodada'] = nrodada
erro_times['Erro_Medio'] = np.round(erro_times['Erro_Medio'],2)
erro_times.to_csv('.\Site\Avaliacao\{}rodada_times.csv'.format(nrodada), index=False, sep='\t', encoding="UTF8")

In [11]:
erro_posicao = pd.DataFrame(df.groupby('Posicao').mean()['Diferenca'])
erro_posicao.reset_index(inplace=True)
erro_posicao.columns = ['Time','Erro_Medio']
#erro_times['Rodada'] = nrodada
erro_posicao['Erro_Medio'] = np.round(erro_posicao['Erro_Medio'],2)
erro_posicao.to_csv('.\Site\Avaliacao\{}rodada_posicao.csv'.format(nrodada), index=False, sep='\t',encoding="UTF8")